##Ubuntu Configurations

###Initial Configuration:

####Enable SSH

    sudo apt-get update
    sudo apt-get install openssh
    [edit /etc/sshd_config]
    sudo service ssh restart

####Enable Power with Lid Down:

    sudo sh -c "echo "HandleLidSwitch=ignore" >> /etc/systemd/logind.conf
    sudo restart systemd-logind  

####Enable manual wireless toggle:

	sudo sh -c "echo \"\niface wlan0 inet manual\" >> /etc/network/interfaces"
	sudo service network-manager restart

####Disable Display Backlight:

    sudo emacs /etc/rc.local
    echo 0 > /sys/class/backlight/*/brightness
        - add to the bottom above "exit 0"
        - replace "*" with actual value on system ("nv_backlight" on macbook)

####Change Hostname:

    sudo sh -c "echo '<new_host_name>' > /etc/hostname"
    ( also change /etc/hosts )
    sudo hostname -f
    
####Copy Over Config Files:

    ~/.bashrc
    ~/.profile
    /etc/profile
    /etc/profile.d/
    ~/SERVER?
    ~/.git/config
      
####Update apt Sources:

    # VIVID LIBRARIES:
    deb http://us.archive.ubuntu.com/ubuntu/ vivid main restricted
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid main restricted
    deb http://us.archive.ubuntu.com/ubuntu/ vivid-updates main restricted
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid-updates main restricted
    deb http://us.archive.ubuntu.com/ubuntu/ vivid universe
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid universe
    deb http://us.archive.ubuntu.com/ubuntu/ vivid-updates universe
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid-updates universe
    deb http://us.archive.ubuntu.com/ubuntu/ vivid multiverse
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid multiverse
    deb http://us.archive.ubuntu.com/ubuntu/ vivid-updates multiverse
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid-updates multiverse
    deb http://us.archive.ubuntu.com/ubuntu/ vivid-backports main restricted universe multiverse
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid-backports main restricted universe multiverse
    deb http://us.archive.ubuntu.com/ubuntu/ vivid-security main restricted
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid-security main restricted
    deb http://us.archive.ubuntu.com/ubuntu/ vivid-security universe
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid-security universe
    deb http://us.archive.ubuntu.com/ubuntu/ vivid-security multiverse
    deb-src http://us.archive.ubuntu.com/ubuntu/ vivid-security multiverse
    #deb http://extras.ubuntu.com/ubuntu vivid main
    #deb-src http://extras.ubuntu.com/ubuntu vivid main


####Install Python 2.7.9 & tools:

    sudo apt-get install python2.7-dev
    sudo apt-get install -y python-setuptools
    wget https://bootstrap.pypa.io/get-pip.py
    sudo python get-pip.py
    rm get-pip.py
    sudo pip install virtualenv
    
####Install Git:

    ap_install git-extras magit
    git clone https://github.com/git/git.git
    echo "CHECK REQUIREMENTS: zlib,ssh,perl,OpenSSL,libcurl,expat,gettext"
    echo `perl -v`" Greater than 5.8?"
    autoconf -i
    ./configure --prefix=/usr --with-tcltk=no
    make all doc
    sudo make install

    
####Install System Tools:

    ap_install htop sshfs lm-sensors cpulimit sysv-rc-conf locate
    sudo sensors-detect
    sudo service kmod start

####Fan Control:

    ap_install macfanctld
    sudo emacs /etc/macfanctl.conf
    sudo service macfanctld restart
    tail -F /var/log/macfanctl.log

####Socat:

    ap_install libwrap0-dev
    wget http://www.dest-unreach.org/socat/download/socat-1.7.2.4.tar.gz
    tar xzvf socat-1.7.2.4.tar.gz
    cd socat-1.7.2.4
    ./configure
    sudo make && sudo make install
    cd ..
    sudo rm -R socat-1.7.2.4.tar.gz socat-1.7.2.4/

## Nginx (OpenResty 1.7.7.1):
####Script Setup:

    source ~/.bashrc
    
    export USER=ub2
    export HOME=/home/$USER
    export SERVER=SERVER2

####Prerequisites:

    sudo apt-get install -y \
        libdrizzle-dev \
        postgresql-server-dev-9.3 \
        libpq-dev \
        libpqxx-dev \
        lua-sql-postgres-dev \
        pgagent \
        postfix-pgsql \
        postgis \
        postgresql-9.3-ip4r \
        postgresql-9.3-plsh \
        postgresql-9.3-postgis-scripts \
        postgresql-client-common \
        postgresql-comparator \
        postgresql-contrib-9.3 \
        postgresql-prioritize \
        postgresql-plpython-9.3 \
        postgresql-plpython3-9.3 || exit 1

####LibDrizzle-1.0

    wget http://openresty.org/download/drizzle7-2011.07.21.tar.gz || exit 1

    tar xzvf drizzle7-2011.07.21.tar.gz || exit 1
    cd drizzle7-2011.07.21/

    sudo apt-get install -y \
        intltool \
        libzthread-dev \
        uuid-dev \
        libreadline-dev \
        libprotobuf-dev \
        protobuf-compiler \
        libprotobuf-dev \
        libboost1.55-all-dev \
        libboost1.55-tools-dev \
        libcurl4-gnutls-dev \
        libgcrypt11-dev || exit 1

    ./configure --without-server && sudo make libdrizzle-1.0 && sudo make install-libdrizzle-1.0 || exit 1


#### OpenResty

####Download latest version:

    wget http://openresty.org/download/ngx_openresty-1.7.7.1.tar.gz || exit 1
    tar -zxvf ngx_openresty-1.7.7.1.tar.gz || exit 1
    cp -R $BD/ipython/notebooks/reference/files/ubuntu_config/mods ngx_openresty-1.7.7.1/ || exit 1
    cd ngx_openresty-1.7.7.1

####Configure

sudo ./configure \
    --with-pcre-jit \
    --with-ipv6 \
    --with-debug \
    --with-luajit \
    --with-http_drizzle_module \
    --with-http_iconv_module \
    --with-libdrizzle=/usr/local \
    --with-http_postgres_module \
    --with-select_module \
    --add-module=mods/nginx-http-rdns-master \
    --add-module=mods/ngx_aws_auth-master \
    --add-module=mods/ngx_http_lower_upper_case-master \
    --add-module=mods/ngx_http_internal_redirect-master \
    --add-module=mods/ngx_upstream_jdomain-master \
    --with-http_auth_request_module \
    --with-http_geoip_module \
    --with-http_realip_module \
    --with-http_sub_module \
    --with-http_addition_module \
    --with-http_spdy_module \
    --with-http_ssl_module \
    --error-log-path=logs/ngx_error.log || exit 1

####Install:

    sudo make && sudo make install || exit 1
    sudo unlink /usr/local/bin/nginx
    sudo ln -s /usr/local/openresty/nginx/sbin/nginx /usr/local/bin/ || exit 1
    cd ..

####Add Lua-resty-logger-socket:

    git clone --recursive https://github.com/cloudflare/lua-resty-logger-socket.git || exit 1
    sudo rm -R /usr/local/openresty/lualib/resty/logger
    sudo mv lua-resty-logger-socket/lib/resty/logger /usr/local/openresty/lualib/resty/ || exit 1

####Add Lua-resty-postgres:

    git clone --recursive https://github.com/azurewang/lua-resty-postgres.git || exit 1
    sudo mv -f lua-resty-postgres/lib/resty/postgres.lua /usr/local/openresty/lualib/resty/ || exit 1
    sudo chown root:root /usr/local/openresty/lualib/resty/postgres.lua
    sudo chmod 644 /usr/local/openresty/lualib/resty/postgres.lua
    sudo rm -R lua-resty-postgres/

####Add Date Functions:

    cd /tmp
    wget http://www.scilua.org/files/time-1.0.0.zip
    sudo unzip time-1.0.0.zip -d /usr/local/openresty/lualib/
    rm time-1.0.0.zip

####Install Luarocks:

    cd /tmp
    wget http://luarocks.org/releases/luarocks-2.0.13.tar.gz
    tar -xzvf luarocks-2.0.13.tar.gz
    cd luarocks-2.0.13/
    ./configure --prefix=/usr/local/openresty/luajit \
        --with-lua=/usr/local/openresty/luajit/ \
        --lua-suffix=jit-2.1.0-alpha \
        --with-lua-include=/usr/local/openresty/luajit/include/luajit-2.1
    make
    sudo make install
    cd ..
    sudo rm -R luarocks*

####Make Luarocks Link:

    sudo ln -s /usr/local/openresty/luajit/luarocks /usr/local/bin

####Add Lua MD5 Library:

    sudo luarocks install md5

####Add LuaSocket:

    sudo luarocks install https://raw.github.com/diegonehab/luasocket/master/luasocket-scm-0.rockspec

####NGX Settings:

    sudo cp -R $HOME/$SERVER/nginx/setup/nginx/conf/nginx.conf $HOME/$SERVER/nginx/
    sudo cp -R $HOME/$SERVER/nginx/setup/nginx/sites-available $HOME/$SERVER/nginx/
    sudo rm -R $HOME/$SERVER/nginx/setup/nginx
    sudo cp -R /usr/local/openresty/nginx  $HOME/$SERVER/nginx/setup
    sudo rm -fR $HOME/$SERVER/nginx/setup/nginx/sites-available

    sudo mv -f $HOME/$SERVER/nginx/nginx.conf $HOME/$SERVER/nginx/setup/nginx/conf/
    sudo mv -f $HOME/$SERVER/nginx/sites-available $HOME/$SERVER/nginx/setup/nginx/

    sudo chown -R $USER:$USER $HOME/$SERVER/nginx
    sudo chmod -R 777 $HOME/$SERVER/nginx

####Clean-up:

    sudo rm -R drizzle7-2011.07.21.tar.gz ngx_openresty-1.7.7.1.tar.gz
    sudo rm -R drizzle7-2011.07.21 lua-resty-logger-socket ngx_openresty-1.7.7.1

## syslog-ng on EC2

### Prerequisites

#####eventlog

	wget https://www.balabit.com/downloads/files/eventlog/0.2/eventlog_0.2.12.tar.gz
	tar -zxvf eventlog_0.2.12.tar.gz
	cd eventlog-0.2.12/
	./configure
	sudo make && sudo make install
	export PKG_CONFIG_PATH=/usr/local/lib/pkgconfig
	cd ..

#####locate (to find eventlog)

	sudo yum install mlocate
	sudo updatedb
	locate eventlog

#####glib2

	sudo yum install glib2-devel.x86_64

#####Support for SQL & pSQL

	sudo yum install libdbi-devel.x86_64 libdbi-dbd-pgsql.x86_64 libdbi-drivers.x86_64

#####Support for SMTP, JSON, and [tcp wrappers](http://www.balabit.com/sites/default/files/documents/syslog-ng-ose-3.3-guides/en/syslog-ng-ose-v3.3-guide-admin-en/html/install_from_source.html)

	sudo yum install libesmtp-devel.x86_64 json-c-devel.x86_64 tcp_wrappers-devel.x86_64


#####Redis ( updating redis from 2.8 to 3.0, see 'redis-server -v' )

	git clone --recursive https://github.com/antirez/redis/archive/3.0.0-rc1.tar.gz
	cd redis-3.0.0-rc1
	make
	sudo yum install tcl-devel.x86_64
	make test
	sudo make install
    sudo sh -c 'echo "vm.overcommit_memory = 1" >> /etc/sysctl.conf'

#####libhiredis
	
	sudo ln -sf /usr/share/zoneinfo/America/New_York /etc/localtime
	git clone --recursive https://github.com/redis/hiredis.git
	cd hiredis
	make && sudo make install
	

### Install	

####syslog-ng

	wget http://www.balabit.com/downloads/files/syslog-ng/open-source-edition/3.6.1/source/syslog-ng_3.6.1.tar.gz
	tar -zxvf syslog-ng_3.6.1.tar.gz 
	cd syslog-ng-3.6.1/
	./configure
	make
	sudo make install

#####clean up

	cd ..
	rm 3.0.0-rc1.tar.gz 
	rm eventlog_0.2.12.tar.gz 
	rm syslog-ng_3.6.1.tar.gz 


### Configuration

#####disable rsyslog

	sudo chkconfig rsyslog off
	sudo service rsyslog stop

####get syslog-ng version

	syslog-ng -V 		# (here, 3.6.1)

####add folder in SERVER4

	cd SERVER4/
	mkdir syslog-ng
	cd syslog-ng

####[from mbp2] copy config file to EC2 folder and revise

	mkdir ~/SERVER4/syslog-ng				
	cp ~/SERVER4/syslog-ng/*.* ~/SERVER4/syslog-ng/
	
	# change version if applicable
	# change "/Users/admin/SERVER3" to "/home/ec2-user/SERVER4"

	cp ~/SERVER4/syslog-ng/*.* /Volumes/ec2/home/ec2-user/SERVER4/syslog-ng/

####update EC2 replace config file

	sudo updatedb
	locate syslog-ng.conf
	sudo mv /usr/local/etc/syslog-ng.conf /usr/local/etc/syslog-ng-tmp.conf
	sudo cp -R ./syslog-ng.conf /usr/local/etc/syslog-ng.conf

#####touch log file

	sudo touch /var/log/nginx.log

#####test configuration

	syslog-ng -evd
	
	

### syslog-ng on Ubuntu 14.04

####Prequisites:

	sudo apt-get install -y autoconf libtool libglib2.0-dev libdbi-dev libdbd-pgsql libdbd-mysql libdbd-sqlite3 || exit 1
	sudo apt-get install -y libesmtp-dev libjson-c-dev libjson-c2 libwrap0-dev libhiredis-dev libssl-dev || exit 1

####Redis:

	wget https://github.com/antirez/redis/archive/3.0.0-rc1.tar.gz || exit 1
	tar -zxvf 3.0.0-rc1.tar.gz
	cd redis
	make && make test && sudo make install || exit 1
    sudo sh -c 'echo "\nvm.overcommit_memory = 1" >> /etc/sysctl.conf'
	cd ..

####Install:

	wget http://www.balabit.com/downloads/files/syslog-ng/open-source-edition/3.6.1/source/syslog-ng_3.6.1.tar.gz
	tar -zxvf syslog-ng_3.6.1.tar.gz 
	cd syslog-ng-3.6.1/
	./configure && make && sudo make install
	cd ..

####Incubator:

	git clone --recursive https://github.com/balabit/syslog-ng-incubator.git
	cd syslog-ng-incubator/
	sudo apt-get install -y bison flex libriemann-client-dev riemann-c-client 
	sudo apt-get install -y libmongo-client-dev lua5.1 librdkafka-dev
    sudo apt-get install -y autoconf libtool
	autoreconf -i
	./configure
    sudo make
    sudo make install
	cd ..
    
####Incubator fo EC2 -- DIDN'T INSTALL !!
	
####Clean-up:

	sudo rm 3.0.0-rc1.tar.gz syslog-ng_3.6.1.tar.gz
	sudo rm -R redis-3.0.0-rc1 syslog-ng-3.6.1/ syslog-ng-incubator/
	

####Install iPython:

    ( copy .ipython to ~/ )
    ( change "==" to ">=" in script )
    ( add pip install:
        cairosvg
        elasticsearch)
    
    ap_install ca-certificates
    ap_install libgeos-dev libgeos++-dev
    ap_install libgcc-4.8-dev gcc-4.8 libcurlpp0
    ap_install python-tables libpq-dev python-pycurl
    ap_install libfreetype6-dev libpng++-dev libpng12-dev
    ap_install postgresql-9.4 postgresql-contrib-9.4 postgresql-9.4-postgis-2.1 postgresql-9.4-postgis-2.1-scripts postgresql-9.4-pgrouting postgresql-9.4-plsh postgresql-9.4-pllua

    cd $HOME/$SERVER/ipython && ./ipy_setup.sh

####[PhantomJS](http://phantomjs.org/build.html) 
    
    wget https://bitbucket.org/ariya/phantomjs/downloads/phantomjs-1.9.8-linux-x86_64.tar.bz2
    tar xvjf phantomjs-1.9.8-linux-x86_64.tar.bz2
    cd phantomjs-1.9.8-linux-x86_64/
    sudo mv bin/phantomjs /usr/local/bin
    cd ..
    rm -R phantomjs-1.9.8-linux-x86_64 phantomjs-1.9.8-linux-x86_64.tar.bz2

#### AWS CLI

    pip install awscli

# see SERVER2/BUILD/

####Upgrade to Vivid:

    

####Correct library issue with syslog-ng:

	sudo ldconfig -v
	
> According to ldconfig manual:
> 
>    ldconfig creates the necessary links  and  cache  to  the  most  recent
>    shared  libraries  found  in  the  directories specified on the command
>    line, in the file /etc/ld.so.conf, and in the trusted directories (/lib
>    and  /usr/lib).  The cache is used by the run-time linker, ld.so or ld-
>    linux.so. 
>    
> So, when you add a new library on Ubuntu, you should run:
> 
>    sudo ldconfig -v 
> where -v is verbose mode


    

sudo mkdir /var/log/syslogs
sudo chown -R ub1:ub1 /var/log/syslogs
sudo chmod -R 777 /var/log/syslogs
touch /var/log/syslogs/supervisor.log

## SERVER 5 - Git Server

In [1]:
!pwd

/home/ub2/SERVER2/BD_Scripts/ipython/notebooks/reference


In [3]:
from IPython.display import FileLinks

FileLinks('./files/BUILD/git_server')

./files/BUILD/git_server/
  chroot.sh
  make_chroot_jail.bash
  sshd_config
  git_config
  ssh_config

#### Setup Chroot Jail

    ./make_chroot_jail.bash
    
#### Make All User Groups

    sudo groupadd all_users;\
    sudo usermod -G all_users -a 'root';\
    sudo usermod -G all_users -a 'ub2';\
    sudo usermod -G all_users -a 'serv'
    # have to log out to effect group changes
    
#### Mount & Bind External HD

    #sudo mount -t hfsplus -o force,defaults,users,umask=0 /dev/sdb2 /Volumes/EXT_HD
    #sudo chown -R serv:serv /Volumes/EXT_HD
    #sudo chmod -R 775 /Volumes/EXT_HD
    #sudo mount -o bind,uid=1002,gid=1005 /Volumes/EXT_HD /home/jail/home/serv/EXT_HD

#### Setup Local Vars

    sudo mkdir /home/jail/home/serv/SERVER5;\
    sudo rm -f /home/jail/home/serv/.bashrc;\
    sudo cp -R ~/SERVER2/local_config /home/jail/home/serv/SERVER5/;\
    sudo ln -s /home/jail/home/serv/SERVER5/local_config/.bashrc /home/jail/home/serv/
    
    [[ emac /etc/passwd and replace '/home/jail/home/serv:' with '/home/jail/home/serv:/bin/bash' ]]

#### Setup SSH id on Remote

    ssh-keygen -t rsa -b 4096
    
    [[ add to github.com ]]

#### Setup Git Server on Remote

    git clone --shared --recursive git@github.com:sethc23/system_config.git

cp SERVER4/.alias_shared SERVER5/
cp SERVER4/.alias_linux SERVER5/
   12  04/02/15 01:18:32  source ~/.bashrc
   13  04/02/15 01:18:40  ls
   14  04/02/15 01:18:45  ls SERVER5
   15  04/02/15 01:18:51  cd SERVER5
   16  04/02/15 01:18:53  mkdir
   17  04/02/15 01:18:58  mkdir local_config
   18  04/02/15 01:19:10  mv ~/.bashrc local_config/
   19  04/02/15 01:19:18  mv ~/.profile local_config/
   20  04/02/15 01:19:45  ln -s local_config/.bashrc ~/
   21  04/02/15 01:19:51  ln -s local_config/.profile ~/
   22  04/02/15 01:19:53  cd ~/
   23  04/02/15 01:19:54  ls
   24  04/02/15 01:20:08  unlink .bashrc
   25  04/02/15 01:20:10  ls
   26  04/02/15 01:20:43  ln -s ~/system_config/SERVER5/local_config/.bashrc ./
   27  04/02/15 01:20:47  ln -s ~/system_config/SERVER5/local_config/.profile ./
   28  04/02/15 01:20:52  unlink .profile
   29  04/02/15 01:20:53  ln -s ~/system_config/SERVER5/local_config/.profile ./

#### Add local_serv on Each Server

    git remote add local_serv serv@serv:/home/jail/home/serv/system_config.git


